In [1]:
from pathlib import Path
import pandas as pd
from function import run_match
# import altair as alt
# import datapane as dp


def clean_text(text):
    return str(text).lower().strip()

In [2]:
path = Path.home() / 'OneDrive - Seagroup/nlp/example/sec__edgar_company_info.csv'
df = pd.read_csv(path).head(50_000)
print(df.shape)

df['company_edit_name'] = df['Company Name'].map(clean_text)
df.head()

(50000, 3)


,Line Number,Company Name,Company CIK Key,company_edit_name
0,1,!J INC,1438823,!j inc
1,2,"#1 A LIFESAFER HOLDINGS, INC.",1509607,"#1 a lifesafer holdings, inc."
2,3,#1 ARIZONA DISCOUNT PROPERTIES LLC,1457512,#1 arizona discount properties llc
3,4,#1 PAINTBALL CORP,1433777,#1 paintball corp
4,5,$ LLC,1427189,$ llc


In [3]:
item = df['company_edit_name'].unique().tolist()

# run
match_df = run_match(base=item, source=item, top=5, similarity=0.6)
match_df['rank'] = match_df.groupby(['base'])['source'].transform(lambda x: pd.factorize(x)[0]).add(1)

process vectorize: 2.68s
process optimized: 5.39s
49,627 skus in BASE match 49,627 skus in SOURCE


In [4]:
match_df.head()

,base,source,similarity,rank
0,!j inc,!j inc,1.00,1
1,"#1 a lifesafer holdings, inc.","#1 a lifesafer holdings, inc.",1.00,1
2,#1 arizona discount properties llc,#1 arizona discount properties llc,1.00,1
3,#1 arizona discount properties llc,arizona income properties llc,0.61,2
4,#1 paintball corp,#1 paintball corp,1.00,1


In [5]:
match_df.query('base == "aim variable insurance funds"')

,base,source,similarity,rank
54039,aim variable insurance funds,aim variable insurance funds,1.00,1
54040,aim variable insurance funds,aim variable insurance funds inc,0.97,2
54041,aim variable insurance funds,aim variable insurance funds (invesco variable...,0.88,3
54042,aim variable insurance funds,aip variable insurance trust,0.68,4
54043,aim variable insurance funds,access variable insurance trust,0.68,5


In [6]:
# dp.Report(
#     dp.DataTable(match_df),
#     match_df.query('base == "aim variable insurance funds"')
# ).upload(name="string matching example")